# Imports

In [ ]:
import logging

import os
import gc
import json
import time
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MaxAbsScaler
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_recall_fscore_support as get_metrics
from sklearn.metrics import roc_auc_score as get_auc

import pipeline_blocks.core_pipeline_function_wrappers
from pipeline_blocks.core_pipeline_function_wrappers import generate_unusable_data_lists
from pipeline_blocks.core_pipeline_function_wrappers import generate_and_reduce_patient_matrix
#from pipeline_blocks.core_pipeline_function_wrappers import generate_features
from utils.pickle_functions import pickle_already_exists, load_pickle
from utils.helpers import print_num_patients_with_SC, extract_patients

from pipeline_blocks.event_matching import print_patient_matrix
from pipeline_blocks.model_training import get_train_and_val_data
from pipeline_blocks.model_training import create_and_compile_model, get_binary_class_occurrences, train_and_evaluate_model, plot_model_histories, preprocess_stft, remove_nan_values_from_dataset, print_class_balance
from pipeline_blocks.feature_extraction import plot_spectrograms

from main_controller import load_or_create_pickle
from importlib import reload

# Parameter definition

In [ ]:
#There are some path issues, make sure the root_path is pointing to the root of "dengue-severity-classification" in order for the rest of the notebook to work
#os.chdir('..')
root_path = os.path.dirname(os.getcwd())
#root_path = ''
print(root_path)
#Define path to 01NVa dataset (point to Imperial RDS drive mounted on your college PC)
base_path = r"R:\projects\vital_project\live\01NVa_Dengue\RAW_DATA"

#Define clinical question and setup appropriate path (adults or children)
event_type = 'severities'
#event_type = 'ICU-FU'
if event_type == 'severities':
    cohort_of_interest = 'adults'
    dataset_path = os.path.join(base_path, "Adults")
elif event_type == 'ICU-FU':
    cohort_of_interest = 'children'
    dataset_path = os.path.join(base_path, "Children")
else:
    raise ValueError("Invalid choice of event type. Pipeline currently thoroughly tested only for ['ICU-FU', 'severities']")

#Singla sampling rate. Smartcare is 100. Only change if working with GE data.
sampling_rate = 100

# this refers to the length of the segments that we examine
# when determining whether a segment is unusable or not
# for reference: 525.6 seconds is the duration of STFT used
unusable_segment_length_seconds = 52.56
unusable_segment_length_samples = int(unusable_segment_length_seconds * sampling_rate)
# used for obtaining the unusable segments, should always 
# be None (non-None values used only for testing)
num_rows_to_read = None


# only important for Shock/Reshock events for now this window refers to the 
# time period that we are interested in before and after an event occurrence
window_size_minutes = 2
window_size_samples = window_size_minutes * 60 * sampling_rate

# only important when working with forecasting dengue shock
pred_horizon_mins = 120
pred_horizon      = pred_horizon_mins * 60 * sampling_rate

# only important for RAW FE method (i.e, using raw signal as features)
# this refers to the length of the segments that we cut the signal 
# up into to produce training samples in the feature extraction stage
fe_segment_length_seconds = 30
fe_segment_length_samples = int(fe_segment_length_seconds * sampling_rate)


# choose which wavelength to use from the two that the Smartcare sensor collects: 'RED' or 'IR'
signal_wavelength = 'RED'

# choose which feature extraction method is applied
# fe_method = 'RAW'
# fe_method = 'FFT'
fe_method = 'STFT'

# this refers to whether we are using an STFT of the full possible length 
# (optimally 10 wavelengths of the lowest frequency of interest) or whether 
# we are using a smaller STFT to increase the number of training examples. 
# The STFT divider dictates how many times smaller the chosen STFT is then the "optimal" one
window_divider = 1


#    num_of_classes = 2
num_of_classes = 2
test_split = 0.2
epochs = 100

filter_params = {
    'type' : 'cheby',
    'order' : 2,
    'sample_rate_Hz' : 100,
    'low_cutoff_Hz' : 0.15,
    'high_cutoff_Hz' : 20
}

sqi_thresholds = {
    'MSQ' : 0.8,
    'zero_cross' : [0.01, 0.04]
}


# Pipeline

## Create file names for the intermediate files

In [ ]:
# define pickle names and locations for the different stages of the pipeline
patient_info_pickle_name = 'patient_info.pkl'
patient_class_segments_pickle_name = 'segmented_classification_physio.pkl'
unusable_data_pickle_name = f'unusable_data_{cohort_of_interest}_{signal_wavelength}_{unusable_segment_length_samples}_{num_rows_to_read}.pkl'
patient_matrix_pickle_name = f'patient_matrix_{cohort_of_interest}_{signal_wavelength}_{event_type}_{window_size_samples}.pkl'

patient_info_pickle_path = os.path.join(root_path, 'data_pickles', 'patient_info', patient_info_pickle_name)
class_segment_pickle_path = os.path.join(root_path, 'data_pickles', 'patient_info', patient_class_segments_pickle_name)
unusable_data_pickle_path = os.path.join(root_path, 'data_pickles', 'unusable_data', unusable_data_pickle_name)
patient_matrix_pickle_path = os.path.join(root_path, 'data_pickles', 'patient_matrix', patient_matrix_pickle_name)


## Load patient information list

In this step a previously created list of all files including clinical information is loaded to guide patient data extraction. 
The file is a python dictonary, storing details about all raw PPG files as well as clinical data.

It is structured as follows:

* PATIENT_ID
    * List of GE recordings
        * Filename
        * Start time
        * End time
        * Duration
    * List of SmartCare recordings
        * Filename
        * Start time
        * End time
        * Duration
    * Vital sign readings
    * Lab readings
    * Fluid readings


In [ ]:
# load patient info pickle
patient_info = load_pickle(patient_info_pickle_path)
#print(patient_info)
print_num_patients_with_SC(patient_info)

In [ ]:
#Switch for more advanced feature extraction based on segment classification
#For none - old operation
#segmented_labels = {} 
segmented_labels = load_pickle(class_segment_pickle_path)

# print(segmented_labels)

trial_lbl = segmented_labels['003-2200']
#print(trial_lbl)
print(len(segmented_labels))
print(trial_lbl)

In [ ]:
# df_vital = pd.read_excel("R:/projects/vital_project/live/01NVa_Dengue/CLINICAL/clinical_data_sheet_copy.xlsx", sheet_name = 2)
df_static  = pd.read_csv(root_path + "/01NVa_clinical_data/output/merged_staticdata.csv")
df_cli     = pd.read_csv(root_path + "/01NVa_clinical_data/output/cleaned_cli.csv")
df_reshock = pd.read_csv(root_path + "/01NVa_clinical_data/output/cleaned_cli_reshock.csv")
df_vital   = pd.read_csv(root_path + "/01NVa_clinical_data/output/cleaned_vital.csv")
df_dengue  = pd.read_csv(root_path + "/01NVa_clinical_data/output/cleaned_dengue_.csv")



check_times = df_static["SUBJID"]
shock_adm   = df_cli["SUBJID"]
id_array    = df_dengue["id"]

for key in segmented_labels:
    
    if(len(segmented_labels[str(key)].keys()) != 0):
        _key_str_ = str(key)
        key_str = _key_str_[4:]
        
        print(key_str)
        for item in segmented_labels[str(key)]:
            shock_obs = []
            timestamp_str = str(item)

            # print(segmented_labels[str(key)][str(item)])
            tmp_date = datetime.datetime.strptime(timestamp_str[:-4], "%Y%m%dT%H%M%S.%f%z")
            start_time_ = time.mktime(tmp_date.timetuple())
            print(tmp_date)
            print(start_time_)

            k_idx = np.where(check_times == int(key_str))[0]
            m_idx = np.where(shock_adm == int(key_str))[0]
            n_idx = np.where(id_array == int(key_str))[0]
            print(k_idx)
            print(m_idx)
            print(n_idx)

            str_datetime = df_static["adm_datetime"][k_idx].values[0]
            if(str_datetime!="-1"):
                ed_date = datetime.datetime.strptime(str_datetime+"+0700", "%d/%m/%Y %H:%M%z")
                print(ed_date)

                ed_start_time = time.mktime(ed_date.timetuple())
                print(ed_start_time)

                df = segmented_labels[str(key)][str(item)]
                df = df.reset_index()

                df['shock_event'] = 0
                df['shock_start_idx'] = df['start_idx']
                df['shock_end_idx']   = df['end_idx']

                for n in n_idx:
                    start_time = df["start_idx"]/100 + start_time_
                    end_time   = df["end_idx"]/100 + start_time_

                    event_time  = df_dengue["t.stop"][n]
                    event_value = df_dengue["event"][n]

                    if(event_value==1):
                        event_time_start_unix = ed_start_time + event_time*3600
                        event_time_end_unix = ed_start_time + event_time*3600 + (10800*2)
                        
                        # print(event_time_start_unix)
                        # print(event_time_end_unix)
                        # print(start_time)
                        # print(end_time)
                        
                        print("shock_event_start")
                        selected_label_1 = df.loc[(start_time <= event_time_start_unix ) & (end_time >= event_time_start_unix)]
                        shock_start_idx  = int(100*(event_time_start_unix - start_time_))
                        print(selected_label_1)

                        print("shock_event_end")
                        selected_label_2 = df.loc[(start_time <= event_time_end_unix ) & (end_time >= event_time_end_unix)]
                        shock_end_idx    = int(100*(event_time_end_unix - start_time_))
                        print(selected_label_2)


                        if(len(selected_label_1) == 0 and len(selected_label_2) > 0):
                            print(selected_label_2.index[0])
                            idx = selected_label_2.index[0]

                            if(idx + 1 != len(df)):
                                df["shock_end_idx"][idx] = shock_end_idx
                                df["shock_start_idx"][idx + 1] = shock_end_idx + 1

                            if(idx > 0):
                                for k in range(0, selected_label_2.index[0] + 1):
                                    df["shock_event"][k] = 1
                                    
                            else:
                                df["shock_event"][idx]   = 1
                                # df["shock_end_idx"][idx] = shock_end_idx
                                # df["shock_start_idx"][idx + 1] = shock_end_idx + 1

                        elif(len(selected_label_1) > 0 and len(selected_label_2) == 0):
                            print(selected_label_1.index[0])

                            idx = selected_label_1.index[0]
                            if(idx > 0):
                                if(selected_label_1.index[0] != 0):
                                    df["shock_end_idx"][idx - 1] = shock_start_idx - 1
                                    df["shock_start_idx"][idx]   = shock_start_idx

                                for k in range(idx, len(selected_label_1) + 1):
                    
                                    df["shock_event"][k] = 1
                                    # if(k == selected_label_1.index[0]):
                                    #     df["shock_end_idx"][k] = shock_end_idx
                                    #     df["shock_start_idx"][k + 1] = shock_end_idx + 1
                            else:
                                df["shock_event"][idx]   = 1
                                # df["shock_end_idx"][idx - 1] = shock_start_idx
                                # df["shock_start_idx"][idx]   = shock_start_idx


                        elif(len(selected_label_1) > 0 and len(selected_label_2) > 0):
                            print(selected_label_1.index[0])
                            print(selected_label_2.index[0])

                            num_idx = selected_label_2.index[0] - selected_label_1.index[0] + 1

                            if(selected_label_2.index[0] + 1 != len(df)):
                                idx = selected_label_2.index[0]
                                df["shock_end_idx"][idx] = shock_end_idx
                                df["shock_start_idx"][idx + 1] = shock_end_idx + 1
                            
                            if(selected_label_1.index[0] != 0):
                                idx = selected_label_1.index[0]
                                df["shock_end_idx"][idx] = shock_end_idx
                                df["shock_start_idx"][idx + 1] = shock_end_idx + 1

                            for k in range(selected_label_1.index[0], selected_label_2.index[0] + 1):
                                df["shock_event"][k] = 1
                                
                
                
                    else:
                        print("no synchronised input")
                
                segmented_labels[str(key)][str(item)] = df.set_index('index')

    else:
        print("no PPG observations")






In [ ]:
trial_lbl = segmented_labels['003-2012']
#print(trial_lbl)
print(len(segmented_labels))
print(trial_lbl)

## Pickle generating functions

From now on, functions are called in a pipeline manner and after each step the results are saved into a Python class-preserving file format called pickle (.pkl)

The function always checks if pickle doesn't already exists and if it does it simply loads it instead of computing to save on subsequent runs

### Filter files

Iterate through all applicable files and determine which segments are bad quality. 

Output is a list containing patient ID, filename and list of starting indexes within the file where noisy segment was identified. 

Each segment is approximately 52 seconds long.


In [ ]:
# check if there already exists a pickle with the unusable data for the current parameters chosen
# currently only accounts for cohort and the length of the segments to be examined for quality checking
# if the pickle exists, load it, if not, generate it
unusable_data = load_or_create_pickle(pickle_path=unusable_data_pickle_path, 
                                        function_to_generate_pickle=generate_unusable_data_lists,
                                        base_path=dataset_path,
                                        unusable_segment_length_samples=unusable_segment_length_samples, 
                                        filter_params=filter_params, 
                                        sqi_thresholds=sqi_thresholds, 
                                        unusable_data_pickle_path=unusable_data_pickle_path,
                                        signal_wavelength=signal_wavelength,
                                        num_rows_to_read=num_rows_to_read)

#print((unusable_data))
print(len(unusable_data)) #224

### Generate patient matrix

Reduce the total number of files to only the relevant one for the clinical question investigated.

In the case of severity classification, all adults files are kept anyway (as long as they have SmartCare data)

In [ ]:
# check if there already exists a pickle with the patient matrix for the current parameters chosen
# currently only accounts for cohort, event type, and the size of the window before and after the event
# if the pickle exists, load it, if not, generate it
reduced_patient_matrix = load_or_create_pickle(pickle_path=patient_matrix_pickle_path, 
                                                function_to_generate_pickle=generate_and_reduce_patient_matrix,
                                                base_path=dataset_path,
                                                patient_info=patient_info,
                                                event_type=event_type,
                                                window_size_samples=window_size_samples,
                                                unusable_data=unusable_data,
                                                unusable_segment_length_samples=unusable_segment_length_samples,
                                                patient_matrix_pickle_path=patient_matrix_pickle_path)
print('Reduced patient matrix:')
print_patient_matrix(reduced_patient_matrix, event_type)
print('\n')

# reduced_patient_matrix = reduced_patient_matrix[0:10]

# print('Reduced patient matrix:')
# print_patient_matrix(reduced_patient_matrix, event_type)
# print('\n')


### Prepare training dataset and define pickle name

In [ ]:
# How long the STFT windows are. Multiplies of ~66 seconds. 

num_stft_windows = 1
#num_stft_windows = 15

stft_resolution = 1
# Resolution of 1 minute STFT window (number of columns for 66 second segment). 1=2columns, 
# Supply list of pts
pt_list = [2053, 2001, 2002]
reduced_patient_matrix_adjusted = extract_patients(reduced_patient_matrix, pt_list)
#print(reduced_patient_matrix_adjusted)
adjust_pt_list = False
#Feature extraction pickle name (change to trigger feature extraction pipeline)

if adjust_pt_list:
    reduced_patient_matrix_input = reduced_patient_matrix_adjusted
else:
    reduced_patient_matrix_input = reduced_patient_matrix

### Generate features

Depending on the type of model and feature extraction method, this step creates feature-label pairs for supervised learning

In [ ]:
#Define unique name for given set of FE parameters
num_of_pts = len(set([row[0][0] for row in reduced_patient_matrix_input]))
classifier = 'physio_score'
training_data_pickle_name = f'feature_extraction_{event_type}_{fe_method}_{num_stft_windows}_{stft_resolution}_{classifier}_{pred_horizon_mins}_mt_1.pkl'
training_data_pickle_path = os.path.join(root_path, 'data_pickles', 'feature_extraction', fe_method, training_data_pickle_name)

print(training_data_pickle_path)

# check if there already exists a pickle with the training data/features for the current parameters chosen
# currently only accounts for cohort, window size, and the size of segments we cut up the signal for FE
# if the pickle exists, load it, if not, generate it
# 0:108    108:216   216:
#reduced_patient_matrix = reduced_patient_matrix[0:2]
feature_extraction_data = load_or_create_pickle(pickle_path=training_data_pickle_path,
                                                function_to_generate_pickle=pipeline_blocks.core_pipeline_function_wrappers.generate_features,
                                                base_path=dataset_path,
                                                event_type=event_type,
                                                reduced_patient_matrix=reduced_patient_matrix_input,
                                                window_size_samples=window_size_samples,
                                                filter_params=filter_params,
                                                signal_wavelength=signal_wavelength,
                                                fe_method=fe_method,
                                                fe_segment_length_samples=fe_segment_length_samples,
                                                window_divider=window_divider,
                                                num_stft_windows=num_stft_windows,
                                                stft_resolution=stft_resolution,
                                                training_data_pickle_path=training_data_pickle_path,
                                                segmented_labels=segmented_labels,
                                                label_name=classifier)




## Model training and evaluation

### Prepare dataset


In [ ]:
print('* Preparing datasets for model training...')
inputs, cs_output, shock_output, patients_id_ = feature_extraction_data
# inputs, cs_output, patients_id_ = feature_extraction_data

if fe_method == 'STFT':
    num_bins = 128
    truncating_point = int(0.4 * len(inputs[0]))
    num_ffts_in_stft = inputs[0].shape[1]
    start_time = time.time()
    inputs = [preprocess_stft(stft, num_bins, truncating_point, num_ffts_in_stft) for stft in inputs]
    end_time = time.time()
    print(f'Preprocessing STFT inputs took {end_time - start_time} seconds')
    # inputs = np.asarray(inputs)

else:
    raise ValueError("Invalid FE method chosen, CNNs are used only for STFT")

_data_ = {'input': inputs, 'score_output':cs_output, 'shock_output':shock_output, 'id':patients_id_}
# _data_ = {'input': inputs, 'score_output':cs_output, 'id':patients_id_}
df = pd.DataFrame(data=_data_)
patients_id = pd.unique(df['id'])

In [ ]:
# df.loc[df['id'] == '2103']

## Generate static clinical and demographic information

In [ ]:
static_data_path = os.path.join(root_path, '01NVa_clinical_data', 'output', 'merged_staticdata.csv')
df_dengue   = pd.read_csv(static_data_path)

features_id = ["SUBJID", "AGE", "WEIGHT", "HEIGHT", "SEX", "HYPER", "DIABETE", "HEADACHE", "VOMIT", "DIARRHOEA", "ABDOPAIN", "SHOCKADM"]

df_static = df_dengue[features_id]

sex_mapping  = {'M':0, 'F':1}
bool_mapping = {'N':0, 'Y':1}


df_static=df_static.replace(to_replace="N",value=0)
df_static=df_static.replace(to_replace="Y",value=1)
df_static=df_static.replace(to_replace="M",value=0)
df_static=df_static.replace(to_replace="F",value=1)


df_static

In [ ]:
df_x = df_static[["AGE", "WEIGHT", "HEIGHT"]]
df_x = df_x.replace(-1, np.nan)

transformer = MaxAbsScaler()
df_x = transformer.fit_transform(df_x)

df_static["AGE"] = df_x[:,0]
df_static["WEIGHT"] = df_x[:,1]
df_static["HEIGHT"] = df_x[:,2]

df_static = df_static.replace(np.nan, -1)

In [ ]:
#creating an array detailing the unique number of patients for creating the subsequent dataset
ids = df_static["SUBJID"].to_numpy()
unique_static_ids = set(ids)

unique_id = set([int(id__) for id__ in patients_id])
print(sorted(unique_id))
print(sorted(unique_id - unique_static_ids))

In [ ]:
#a routine for creating the spectrogram image sequences 
seq_len = 10
_unique_patient_ids_ = pd.unique(df['id'])
arrX, arrY1, arrY2, arrID = [], [], [], []
uniqueID = []
for _id_ in _unique_patient_ids_:
    _arr_   = df.loc[df['id'] == _id_]['input'].reset_index(drop=True)
    _label_1_ = df.loc[df['id'] == _id_]['score_output'].reset_index(drop=True)
    _label_2_ = df.loc[df['id'] == _id_]['shock_output'].reset_index(drop=True)
    if(len(_arr_) >= seq_len):
        arr = np.array([x for x in _arr_])
        for k in range(seq_len, len(arr)): #1
            arrX.append(arr[k-seq_len:k])
            arrY1.append(_label_1_[k-1])
            arrY2.append(_label_2_[k-1])
            arrID.append(_id_)
            
arrX = np.array(arrX)
arrY1 = np.array(arrY1)
arrY2 = np.array(arrY2)
arrID = np.array(arrID)

In [ ]:
#detailing the distribution of labels for both tasks

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import StratifiedGroupKFold

from sklearn.utils.multiclass import type_of_target
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt

label_encoder = LabelEncoder()
print(type_of_target(arrY1))
print(type_of_target(arrY2))
y1 = label_encoder.fit_transform(arrY1)
y2 = label_encoder.fit_transform(arrY2)
seed = np.random.randint(1,1000)

print("Label distribution for binary clinical score")
_ = plt.hist(y1, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram with 'auto' bins")
plt.show()

print("Label distribution for shock prediction")
_ = plt.hist(y2, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram with 'auto' bins")
plt.show()

In [ ]:
# Detect and discard NaN entries
if event_type == 'severities':
    inputs_train_contain_nans = np.isnan(arrX).any()
    if inputs_train_contain_nans:
        print('Input data contains nan values, removing now...')
        arrX1, arrY1, arrID1 = remove_nan_values_from_dataset(arrX, arrY1, arrID)
        arrX2, arrY2, arrID2 = remove_nan_values_from_dataset(arrX, arrY2, arrID)
        arrX  = arrX1
        arrID = arrID1 
    else:
        print('Input data does_train not contain nan values, all good!')
    print_class_balance(arrX, arrY1)
    print_class_balance(arrX, arrY2)

In [ ]:

df_static = df_static.drop_duplicates()
patients_id = set(arrID)
timestep = seq_len
no_of_columns = len(df_static.columns) - 1
static_array = []

for k in range(len(arrID)):
    print(arrID[k])
    _static_query_ = df_static.loc[df_static['SUBJID'] == int(arrID[k])]
    static_query_ = _static_query_.to_numpy()[0][1:]
    static_query = np.broadcast_to(static_query_, (timestep,no_of_columns))
    static_array.append(static_query)

static_input_ = np.array(static_array)
static_input  = static_input_.astype(float)

### Setup model for training and evaluation

In [ ]:
timestep = arrX[0].shape[0]
spectrogram_height = arrX[0].shape[1]
spectrogram_width = arrX[0].shape[2]
input_dim_ = static_input[0].shape[1]
output_dim_ = 1

In [ ]:
def build_cnn(input_shape, number_of_classes):

    kernel_size = (4, 4)
    max_pooling_dims = (2, 2)

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=input_shape))

    model.add(tf.keras.layers.Conv2D(4, kernel_size, activation=tf.keras.layers.LeakyReLU(), padding='same'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.MaxPooling2D(max_pooling_dims, strides=max_pooling_dims, padding='same'))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Conv2D(8, kernel_size, activation=tf.keras.layers.LeakyReLU(), padding='same'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.MaxPooling2D(max_pooling_dims, strides=max_pooling_dims, padding='same'))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Conv2D(16, kernel_size, activation=tf.keras.layers.LeakyReLU(), padding='same'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.MaxPooling2D(max_pooling_dims, strides=max_pooling_dims, padding='same'))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Conv2D(32, kernel_size, activation=tf.keras.layers.LeakyReLU(), padding='same'))
    #model.add(Dropout(0.2))
    model.add(tf.keras.layers.MaxPooling2D(max_pooling_dims, strides=max_pooling_dims, padding='same'))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Conv2D(32, kernel_size, activation=tf.keras.layers.LeakyReLU(), padding='same'))
    #model.add(Dropout(0.2))
    model.add(tf.keras.layers.MaxPooling2D(max_pooling_dims, strides=max_pooling_dims, padding='same'))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(32))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(32))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Dense(16))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Dense(8))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    return model

def build_cnnlstm(input_shape, number_of_classes):

    kernel_size = (4, 4)
    max_pooling_dims = (2, 2)

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=input_shape))

    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(4, kernel_size, activation=tf.keras.layers.LeakyReLU(), padding='same')))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dropout(0.1)))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D(max_pooling_dims, strides=max_pooling_dims, padding='same')))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization()))

    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(8, kernel_size, activation=tf.keras.layers.LeakyReLU(), padding='same')))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dropout(0.1)))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D(max_pooling_dims, strides=max_pooling_dims, padding='same')))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization()))

    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(16, kernel_size, activation=tf.keras.layers.LeakyReLU(), padding='same')))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dropout(0.2)))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D(max_pooling_dims, strides=max_pooling_dims, padding='same')))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization()))

    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(32, kernel_size, activation=tf.keras.layers.LeakyReLU(), padding='same')))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D(max_pooling_dims, strides=max_pooling_dims, padding='same')))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization()))

    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(32, kernel_size, activation=tf.keras.layers.LeakyReLU(), padding='same')))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D(max_pooling_dims, strides=max_pooling_dims, padding='same')))
    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.BatchNormalization()))

    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))
    model.add(tf.keras.layers.LSTM(32))
    model.add(tf.keras.layers.Dense(32))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(32))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Dense(16))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Dense(8))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    return model



### Final averaged metrics + plot

### Perfom K-fold cross validation

In [ ]:
def metrics(prob_pos, y, threshold_cp=0):
    """
    Takes predictions and compares with truth labels
    """
    import numpy as np
    import seaborn as sns
    from sklearn.metrics import roc_curve
    from sklearn.metrics import precision_recall_curve, auc
    from sklearn.metrics import precision_score
    from sklearn.metrics import brier_score_loss
    from sklearn.metrics import roc_auc_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import average_precision_score
    fpr, tpr, thresholds = roc_curve(y, prob_pos)
    J = tpr - fpr
    if(threshold_cp!=0):
        best_thresh = threshold_cp
    else:
        ix = np.argmax(J)
        best_thresh = thresholds[ix]
        #best_thresh = 0.5
    print('Best Threshold=%f' % (best_thresh))
    #Output metrics
    precision, recall, prc_threshold = precision_recall_curve(y, prob_pos)
    
    from sklearn.metrics import classification_report
    
    pred_class = prob_pos > best_thresh
    print(classification_report(y,pred_class))
    from sklearn.metrics import confusion_matrix
    sns.heatmap(confusion_matrix(y,pred_class), annot=True, fmt='g',cmap='Blues')

    tn, fp, fn, tp = confusion_matrix(y,pred_class).ravel()
    specificity = tn/(tn+fp)

    from sklearn.metrics import recall_score
    from sklearn.metrics import make_scorer
    specificity_score = make_scorer(recall_score, pos_label=0)
    npv = make_scorer(precision_score,pos_label=0)

    metrics = {'roc_auc':'roc_auc',
            'specificity':specificity_score,
            'recall':'recall',
            'precision/PPV':'precision',
            'accuracy':'accuracy',
            'npv':npv}

    bs=brier_score_loss(y, prob_pos)
    print('Average precision score', average_precision_score(y, prob_pos))
    print('F1 Score',f1_score(y,pred_class))
    print('Specificity', specificity)
    sensitivity = tp/(tp+fn)
    print('Sensitivity', sensitivity)
    print('ROC score', roc_auc_score(y, prob_pos))
    print('PRC score', auc(recall, precision))

    print('PPV',tp/(tp+fp))
    print('NPV',tn/(fn+tn))
    print('BS',bs)

def individual_metrics(prob_pos, y):
    import numpy as np
    import seaborn as sns
    from sklearn.metrics import roc_curve
    from sklearn.metrics import precision_recall_curve, auc
    from sklearn.metrics import precision_score, recall_score
    from sklearn.metrics import brier_score_loss
    from sklearn.metrics import roc_auc_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import classification_report
    from sklearn.metrics import average_precision_score
    fpr, tpr, thresholds = roc_curve(y, prob_pos)
    J = tpr - fpr
    ix = np.argmax(J)
    best_thresh = thresholds[ix]
    
    print('Best Threshold=%f' % (best_thresh))
    pred_class = prob_pos>best_thresh
    print(classification_report(y,pred_class))
    #Output metrics
    print('Average precision score', average_precision_score(y, prob_pos))
    print('F1 Score', f1_score(y,pred_class))
    print('Precision', precision_score(y,pred_class))
    print('Recall', recall_score(y,pred_class))
    # print('ROC score', roc_auc_score(y, prob_pos))


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import StratifiedGroupKFold

from sklearn.utils.multiclass import type_of_target
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt

label_encoder = LabelEncoder()
print(type_of_target(arrY1))
print(type_of_target(arrY2))
y1 = label_encoder.fit_transform(arrY1)
y2 = label_encoder.fit_transform(arrY2)
seed = np.random.randint(1,1000)

_ = plt.hist(y2, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram with 'auto' bins")
plt.show()

In [ ]:
_ = plt.hist(y1, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram with 'auto' bins")
plt.show()

In [ ]:
model_type      = 'CNN' #model type is either CNN or CNNLSTM
experiment_num  = 1
save_filepath   = root_path + "/results/"+str(pred_horizon_mins)+"/experiment_"+str(experiment_num)+"/"+model_type+"/"

In [ ]:
for idx in range(1,6):
    # idx = 5

    train_idx_ = pd.read_csv(save_filepath + "idx_set/train_idx_"+str(idx)+".csv")
    train_idx = np.squeeze(train_idx_.values)
    train_idx = train_idx.astype(np.int64)

    val_idx_ = pd.read_csv(save_filepath + "idx_set/val_idx_"+str(idx)+".csv")
    val_idx = np.squeeze(val_idx_.values)
    val_idx = val_idx.astype(np.int64)

    test_idx_ = pd.read_csv(save_filepath + "idx_set/test_idx_"+str(idx)+".csv")
    test_idx = np.squeeze(test_idx_.values)
    test_idx = test_idx.astype(np.int64)

    a = arrID[train_idx]
    b = arrID[val_idx]
    c = arrID[test_idx]


    print('Training set')
    display(len(set(a)))
    print(set(a))
    print('Size of training set',len(a))
    print('Validation set')
    display(len(set(b)))
    print(sorted(set(b)-set(a)))
    print(sorted(set(b)-set(c)))
    print('Size of val set',len(b))
    print('Test set')
    display(len(set(c)))
    print(sorted(set(c)-set(a)))
    print('Size of test set',len(c))

In [ ]:
count = 1

cv= StratifiedGroupKFold(n_splits=8, shuffle=False, random_state=seed)
for train_split, test_idx in cv.split(arrX, y2, arrID):
    print("TRAIN:", train_split, "TEST:", test_idx)

X_train_main_ = arrX[train_split,:,:,:,:]
X_train_aux_  = static_input[train_split,:,:]
y_train_aux_  = y1[train_split]
y_train_main_ = y2[train_split]
train_pID     = arrID[train_split]

#and now to split to train/val by repeating the above again but on the train_idx sets 
cv = StratifiedGroupKFold(n_splits=5, shuffle=False, random_state=seed)

for train_idx, val_idx in cv.split(X_train_main_, y_train_aux_, train_pID):
    print("TRAIN:", train_idx, "VAL:", val_idx)

    np.savetxt(save_filepath + "idx_set/train_idx_" + str(count) + ".csv", train_split[train_idx], delimiter=",")
    np.savetxt(save_filepath + "idx_set/val_idx_" + str(count) + ".csv", train_split[val_idx], delimiter=",")
    np.savetxt(save_filepath + "idx_set/test_idx_" + str(count) + ".csv", test_idx, delimiter=",")

    count = count + 1

In [ ]:
#conformal prediction settings
enable_cp = 0
alpha = 0.3

# Model Training

1. The 5 models are trained from the 5 CV folds and tested on the independent test set.
2. At this stage, the imbalanced data in the dengue shock labels are randomly re-sampled in order to balance the dataset. 
3. [Optional] The application of conformal prediction to control the level of Type I errors.

In [ ]:
load_model      = 1
enable_mtl      = 1

for idx in range(1,6):
    # idx = 5

    train_idx_ = pd.read_csv(save_filepath + "idx_set/train_idx_"+str(idx)+".csv")
    train_idx = np.squeeze(train_idx_.values)
    train_idx = train_idx.astype(np.int64)

    val_idx_ = pd.read_csv(save_filepath + "idx_set/val_idx_"+str(idx)+".csv")
    val_idx = np.squeeze(val_idx_.values)
    val_idx = val_idx.astype(np.int64)

    test_idx_ = pd.read_csv(save_filepath + "idx_set/test_idx_"+str(idx)+".csv")
    test_idx = np.squeeze(test_idx_.values)
    test_idx = test_idx.astype(np.int64)

    a = arrID[train_idx]
    b = arrID[val_idx]
    c = arrID[test_idx]


    print('Training set')
    display(len(set(a)))
    print('Size of training set',len(a))
    print('Validation set')
    display(len(set(b)))
    print(sorted(set(b)-set(a)))
    print(sorted(set(b)-set(c)))
    print('Size of val set',len(b))
    print('Test set')
    display(len(set(c)))
    print(sorted(set(c)-set(a)))
    print('Size of test set',len(c))

    #Confirm exclusivity of patient data in train/val/test sets
    print(a.tolist() in b.tolist())
    print(b.tolist() in c.tolist())

    X_train_main = arrX[train_idx,:,:,:,:]
    X_train_aux  = static_input[train_idx,:,:]
    X_val_main   = arrX[val_idx,:,:,:,:]
    X_val_aux    = static_input[val_idx,:,:]
    X_test_main  = arrX[test_idx,:,:,:,:]
    X_test_aux   = static_input[test_idx,:,:]

    y_train_aux   = y1[train_idx]
    y_train_main  = y2[train_idx]
    y_val_aux     = y1[val_idx]
    y_val_main    = y2[val_idx]
    y_test_aux    = y1[test_idx]
    y_test_main   = y2[test_idx]

    bool_train_labels = y_train_main != 0


    display(pd.DataFrame(y_train_main).value_counts(normalize=True))
    display(pd.DataFrame(y_train_aux).value_counts(normalize=True))
    display(pd.DataFrame(y_val_main).value_counts(normalize=True))
    display(pd.DataFrame(y_val_aux).value_counts(normalize=True))
    display(pd.DataFrame(y_test_main).value_counts(normalize=True))

    main_pos_features = X_train_main[bool_train_labels]
    main_neg_features = X_train_main[~bool_train_labels]
    aux_pos_features = X_train_aux[bool_train_labels]
    aux_neg_features = X_train_aux[~bool_train_labels]

    pos_labels = y_train_main[bool_train_labels]
    neg_labels = y_train_main[~bool_train_labels]

    aux_pos_labels = y_train_aux[bool_train_labels]
    aux_neg_labels = y_train_aux[~bool_train_labels]

    ids = np.arange(len(main_pos_features))
    choices = np.random.choice(ids, len(main_neg_features))

    res_pos_features = main_pos_features[choices]
    res_pos_labels   = pos_labels[choices]
    aux_res_pos_features = aux_pos_features[choices]

    aux_res_pos_labels   = aux_pos_labels[choices]


    resampled_features = np.concatenate([res_pos_features, main_neg_features], axis=0)
    resampled_labels = np.concatenate([res_pos_labels, neg_labels], axis=0)
    aux_resampled_features = np.concatenate([aux_res_pos_features, aux_neg_features], axis=0)

    aux_resampled_labels = np.concatenate([aux_res_pos_labels, aux_neg_labels], axis=0)

    order = np.arange(len(resampled_labels))
    np.random.shuffle(order)
    resampled_features = resampled_features[order]
    resampled_labels = resampled_labels[order]
    aux_resampled_features = aux_resampled_features[order]

    aux_resampled_labels = aux_resampled_labels[order]



    if(load_model == 0):

        if(model_type == 'CNN'):
            tmp_x_train =  resampled_features[:,-1,:,:,:]
            X_val_main  =  X_val_main[:,-1,:,:,:]
            X_test_main =  X_test_main[:,-1,:,:,:]
            # X_test_aux  =  X_test_aux[:,-1,:,:,:]

            opt = tf.keras.optimizers.Adam(learning_rate=3e-4)
            input_shape = tmp_x_train[0].shape
            output_dim = 1

            modelSFT = build_cnn(input_shape, output_dim)
        else:
            tmp_x_train =  resampled_features
            X_val_main  =  X_val_main
            X_test_main =  X_test_main
            # X_test_aux  =  X_test_aux[:,-1,:,:,:]

            opt = tf.keras.optimizers.Adam(learning_rate=3e-4)
            input_shape = tmp_x_train[0].shape
            output_dim = 1

            modelSFT = build_cnnlstm(input_shape, output_dim)
            
        if(idx == 1):
            print(modelSFT.summary())

        modelSFT.compile(loss='binary_crossentropy',
                          optimizer=opt,
                          metrics=tf.keras.metrics.AUC(name='auc'))
        
        with tf.device('/cpu:0'):
            history = modelSFT.fit(tmp_x_train, 
                                aux_resampled_labels, 
                                validation_data=(X_val_main, y_val_aux),   
                                batch_size=32, 
                                epochs=30, 
                                callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=30, restore_best_weights=False)])
        
        modelSFT.save(save_filepath + "models/modelSFT_" + str(idx) + ".h5")

    else:
        modelSFT = tf.keras.models.load_model(save_filepath + 'models/modelSFT_' + str(idx) + '.h5')

    if(enable_cp):
        cal_pred_shock = modelSFT.predict(X_val_main)

        n = len(y_val_main)
        normal = y_val_main == 0
        c_preds_normal = cal_pred_shock[normal]
        c_preds_shock  = cal_pred_shock[np.invert(normal)]

        cal_scores = c_preds_normal

        # Use the outlier detection method to get a threshold on the toxicities
        qhat = np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n)
        

        predictions_cs= modelSFT.predict(X_test_main)

        normal = y_test_main == 0
        preds_normal = predictions_cs[normal]
        preds_shock  = predictions_cs[np.invert(normal)]
        # Perform outlier detection on the ind and ood data
        outlier_ind = preds_normal > qhat # We want this to be no more than alpha on average
        outlier_ood = preds_shock > qhat # We want this to be as large as possible, but it doesn't have a guarantee

        # Calculate type-1 and type-2 errors
        type1 = outlier_ind.mean()
        type2 = 1-outlier_ood.mean()
        print(f"The type-1 error is {type1:.4f}, the type-2 error is {type2:.4f}, and the threshold is {qhat:.4f}.")

        print("RESULT " + str(idx))
        # metrics(predictions_shock, y_test_main)
        metrics(predictions_cs, y_test_aux, qhat)
        for id in set(c):
            __idx__ = np.where(c == id)[0]
            print(str(id))
            individual_metrics(predictions_cs[__idx__], y_test_aux[__idx__], qhat)
    else:
        predictions_cs = modelSFT.predict(X_test_main)

        print("RESULT " + str(idx))
        # metrics(predictions_shock, y_test_aux)
        metrics(predictions_cs, y_test_main)